<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 10.2 Machine Learning 3 - Real-time Sound Recognition for Classification

## CNN model deployment to Raspberry Pi


## 2.1 TensorFlow Lite

Based on the sample code in the previous section, let's apply the CNN model to predict the operational state of the vacuum pump. To do this, we will use TensorFlow Lite. TensorFlow Lite has limited features but is faster and lighter than TensorFlow. When you install TensorFlow on Raspberry Pi and laptop, TensorFlow Lite module is automatically installed. Because it supports Keras interface, TensorFlow Lite is able to load and run the CNN model developed in the prelab using TensorFlow and Keras.

The best way to practice implementation of the trained model is to program yourself. Because of the limited time in the lab, however, the complete sample code ([lab10_sample2.py]()) is given. Perform Task 2.1 to interpret the model output as the appropriate label. Note that the CNN model (h5 format) must be in the same directory in which the sample code is.

<br>

### Task 2.1

Run [lab10_sample2.py]() on your Raspberry Pi or laptop see if the CNN model works well. The terminal or output shell looks like as Figure 5. **Note that if you develop the CNN model with different feature size and shape, you have to modify the sample code.** An algorithm to interpret the CNN model output is incomplete as part of the sample code below. Complete the algorithm so that appropriate label is printed out at the end. To be specific, if Y (prediction inference index) is 0, the prediction label must be "OFF" (given) and if Y is 1, the prediction label must be "ON-airleaking", and so on.

* Hint: Use if, elif, else statement.
* Hint: Another way is to use dictionary data structure with indexing.

<br>

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python3 (Last part of [lab10_sample2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample1.py?raw=true))**

```
91         if Y == 0: # if Y is 0, do below
92             prediction_label = "OFF"
93         # complete your algorithm to take appropriate prediction_label string according to the CNN model result
94 
95         print('The pump is now {}.\n'.format(prediction_label))
96         
97         time.sleep(1) # pause for 1 second

```

---

<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig5.png?raw=true" width="90%">

*Figure 5 Terminal while running lab10_sample2.py*

In [ ]:
## copy and paste your entire algorithm from line number 91 (if Y == 0: # if Y is 0, do below) to the end



##Learning goals

Students will be able to:

1. Understand and utilize MTConnect sound data stream
2. Implement a CNN model to predict operational state of the vacuump pump
3. Build a real-time web-based monitoring system

## 1.1 Introduction

In this lab, we will implement real-time machine sound recognition using a CNN model and MTConnect sound stream to predict the operational state of the vacuum pump. This lab is broken down into three main sections: **1) Utilize MTConnect sound data stream**, **2) Implement a CNN model you developed in the prelab using TensorFlow Lite**, and **3) Build up the entire monitoring system.** 

To get started, Figure 1 illustrates a simplified sound recognition system. IIoT technologies and embedded machine learning models realize this system. The idea is that the Raspberry Pi listens to sound from the machine and recognize the operational state by AI model, and then we are able to monitor remotely the real-time state via a web-based dashboard. 

<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig1.png?raw=true" width="70%">

*Figure 1 Simplified system configuration of vacuum pump sound recognition for Lab 10*

<br>

Details of the system configuration and data flow are shown in Figure 2. Please take a close look at Figure 2. Let's say the MTConnect stream for sound is "sub-MTConnect" for sound to distinguish it from the MTConnect for results. First, the internal sound sensor connected to the headless computer captures analog sound signals and then converts them to digital signals. Second, the sub-MTConnect adapter samples, formats, and converts them to the sub-MTConnect agent. Therefore, as long as a client is accessible via TCP/IP to the headless computer, it can read the sound stream using the sub-MTConnect agent in real-time. Third, the edge computer (Raspberry Pi) gets the sound stream and takes a signal processing for the input feature of the CNN model that we developed in Prelab 10. For AI implementation, we utilize TensorFlow Lite on Raspberry Pi. Fourth, the outputs of the CNN model with the AI algorithm are taken to MTConnect. Fifth, the MTConnect data collector program aggregates MTConnect agent data and sends them to the MySQL database. Lastly, Grafana dashboard with the MySQL database can let end users know the current/history running state of the vacuum pump.

<br>


<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig2.png?raw=true" width="100%">

*Figure 2 Detailed system configuration and data flow for Lab 10*

## 1.2 Sensor and server information

Here, sensor specifications and server information (for MySQL and Grafana services) are summarized for your convenience, even though you are able to find the information in the previous prelab and lab manuals.

Sensor specifications including the sub-MTconnect sound stream are in Table 1. [Sound pressure level](https://en.wikipedia.org/wiki/Sound_pressure) (dB SPL) is calculated by the sensitivity, SNR, gain of PCM (pulse-code modulation) in ALSA (advanced Linux sound architecture), and reference pressure (2×10<sup>-5</sup>  Pa). For the details of the conversion, please go to [reference 1](https://en.wikipedia.org/wiki/Decibel), and [reference 2](https://www.analog.com/en/analog-dialogue/articles/understanding-microphone-sensitivity.html#:~:text=The%20microphone's%20dynamic%20range%20is,a%2015%2Dbit%20data%20word.). The gain of PCM is +28.78 dB. Using the information of the sensor and sound stream, perform Task 1.1 to get access to the MTConnect sound stream.

*Table 1 Sound sensor and stream specifications*

<table>
<thead>
  <tr>
    <th>Target</th>
    <th>Specification</th>
    <th>Value</th>
    <th>Unit/remark</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td rowspan="8">USB Microphone</td>
    <td>Sensitivity</td>
    <td>-38±3</td>
    <td>dB</td>
  </tr>
  <tr>
    <td>Output impedance</td>
    <td>680</td>
    <td>Ω</td>
  </tr>
  <tr>
    <td>Directivity</td>
    <td>Unidirectional</td>
    <td>-</td>
  </tr>
  <tr>
    <td>Channel</td>
    <td>Mono</td>
    <td>-</td>
  </tr>
  <tr>
    <td>Frequency range</td>
    <td>20 - 24000</td>
    <td>Hz</td>
  </tr>
  <tr>
    <td>Operation voltage</td>
    <td>5</td>
    <td>V</td>
  </tr>
  <tr>
    <td>Current consumption</td>
    <td>0.5</td>
    <td>mA</td>
  </tr>
  <tr>
    <td>SNR (Signal to Noise Ratio)</td>
    <td>56</td>
    <td>dB</td>
  </tr>
  <tr>
    <td rowspan="8">Sound stream</td>
    <td>Middleware</td>
    <td>MTConnect</td>
    <td></td>
  </tr>
  <tr>
    <td>Agent IP address</td>
    <td>192.168.1.4</td>
    <td>Router network (DNS: mepotrb16.ecn.purdue.edu)</td>
  </tr>
  <tr>
    <td>Port</td>
    <td>5001</td>
    <td></td>
  </tr>
  <tr>
    <td>Data type</td>
    <td>DiscreteTimeseries</td>
    <td></td>
  </tr>
  <tr>
    <td>Data format</td>
    <td>Signed 16-int</td>
    <td>Full scale is -1 to 1.</td>
  </tr>
  <tr>
    <td>Sampling frequency</td>
    <td>48</td>
    <td>kHz</td>
  </tr>
  <tr>
    <td>Count/sequence</td>
    <td>2048</td>
    <td>Approximately 43 msec</td>
  </tr>
  <tr>
    <td>dataitemId</td>
    <td>sensor1</td>
    <td></td>
  </tr>
</tbody>
</table>

### Task 1.1

Using [curl](https://curl.se/) command below on Raspberry Pi and laptop, read current sound stream of the sensor and then attach it to the report as follows. Check the timestamp indicates current. It must be working on your laptop as well as long as both computers are on the same router network. Perform the same in the command prompt of your laptop as Raspberry Pi.


---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal AND <img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Windows.png?raw=tru" width="20"> Windows - Command Prompt**

`curl 192.168.1.4:5001`

OR

`curl mepotrb16.ecn.purdue.edu:5001`

---



*   Note that the headless computer is connected to the router network (DNS: mepotrb16,ecn.purdue.edu). Because of the port forwarding of 5001 to the headless computer (192.168.1.4), we can has the same results if you get access both IP address and DNS.



<br>
<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_img1.png?raw=true" width="100%">





---

Please your capture here (Raspberry Pi)

---





---

Please your capture here (Laptop)

---


## 1.3 Handling MTConnect sound stream in Python

Let's get access to the MTConnect sound stream and handle it using Python. Use [lab10_sample1.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample1.py?raw=true) which is almost complete other than some constant variables. In the Python script, We will read approximately 1-second-long data at the moment for the CNN model we developed in the prelab. And then, signal processing and sound pressure level calculation will be made. Eventually, we will check the sound signal by plotting them.

The incomplete part of the sample script is below. You have to change the variables which is wrarpped by aserteric mark (*). By completing the sample code , perform Task 1.2.

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python3 ([lab10_sample1.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample1.py?raw=true))**

```
## == CONSTANT ==
SAMPLE = "sample" # sample string for MTConnect HTML sample method
CURRENT = "current" # current string for MTConnect HTML current method
*SAMP_RATE* = int(?) # sampling rate
*CHUNK* = int(?) # chunk size
*AGENT* = "http://?ip?:?port?/" # MTConnect agent ip:port

*N* = int(?) # number of sequence to take sound
*N_FFT* = int(?) # number of FFT
*N_MELS* = int(?) # number of Mel filter bank

```


### Task 1.2

To complete lab10_sample1.py, the required information is below. Note that you can use either Raspberry Pi or your laptop for this task.

*   MTConnect (sound stream) agent IP: 192.168.1.4
*   MTConnect agent port number: 5001
*   Sampling frequency (sampling rate): MTConnect sound stream specification
*   Chunk size: MTConnect sound stream specification
*   Number of the required sequence: Same as CNN model training to make the sound signal approximately 1-second-long
*   Number of FFT data points: Same as the CNN model training
*   Number of Mel filter-bank: Same as the CNN model training

<br>

1. After completing it, plot the signal as Figure 3 and then attach it on the report.

2. Calculate and print out the exact signal length in the second unit as Figure 4 and then attach the screenshot. You can use the commented-out line (line 57), but you have to replace "?" to get the answer.


<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig3.png?raw=true" width="90%">

*Figure 3 Sound signal plot: time domain (left) and Mel-spectrogram (right)*

<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig4.png?raw=true" width="80%">

*Figure 4 Terminal after running lab10_sample1.py*




---

Place your screenshot for 1 of Task 1.2.

---






---

Place your screenshot for 2 of Task 1.2.

---



<br></br>

Please continue to [Lab 9.2 here](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab2.ipynb).
